# Xarray Tile Testing

In [5]:
import boto3
import pandas as pd
import json
from xarray_tile_test import XarrayTileTest
import sys; sys.path.append('..')
import helpers.eodc_hub_role as eodc_hub_role
import warnings
warnings.filterwarnings("ignore")

In [2]:
credentials = eodc_hub_role.fetch_and_set_credentials()

In [3]:
xarray_tile_test = XarrayTileTest(
    dataset_id='cmip6-kerchunk',
    dataset_url='s3://nasa-eodc-data-store/test-data/cmip6-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json',
    variable='tas',
    extra_args={'reference': True},
)

In [ ]:
xarray_tile_test.array_specs

In [ ]:
xarray_tile_test.run_batch({'zoom': 0})

In [ ]:
xarray_tile_test.store_results(credentials)

## Run many tests

In [3]:
# TODO
zooms = range(12)
target_datasets = json.loads(open('../01-generate-datasets/all-datasets.json').read())
list(target_datasets.items())[0:2]

[('cmip6-kerchunk',
  {'dataset_url': 's3://nasa-eodc-data-store/test-data/cmip6-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json',
   'variable': 'tas',
   'extra_args': {'reference': True}}),
 ('power_901_monthly_meteorology_utc.zarr',
  {'dataset_url': 's3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr',
   'variable': 'TS'})]

In [4]:
%%time
result_files = []
for dataset_id, dataset in target_datasets.items():
    xarray_tile_test = XarrayTileTest(
        dataset_id=dataset_id,
        **dataset
    )
    xarray_tile_test.run_batch({'zoom': 0})
    result_file = xarray_tile_test.store_results(credentials)
    result_files.append(result_file)

Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231646_XarrayTileTest_cmip6-kerchunk.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231647_XarrayTileTest_power_901_monthly_meteorology_utc.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231648_XarrayTileTest_cmip6-pds_GISS-E2-1-G_historical_tas.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231651_XarrayTileTest_aws-noaa-oisst-feedstock_reference.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231652_XarrayTileTest_600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231653_XarrayTileTest_600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230831231655_XarrayTileTest_365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results

## Read results

In [6]:
def load_all_into_dataframe(s3files: list[str]):
    boto3_session = boto3.Session(**credentials)
    s3_client = boto3_session.client('s3')
    dfs = []

    for s3url in s3files:
        df = pd.read_json(s3url, orient='index').T
        dfs.append(df)
 
    merged_df = pd.concat(dfs)
    merged_df.set_index('dataset_id', inplace=True)
    return merged_df


In [7]:
load_all_into_dataframe(result_files)

,test_name,dataset_url,niters,variable,extra_args,timings,reference,lat_extent,lon_extent,number_coordinate_chunks,total_array_size,chunks,shape_dict,dtype,chunk_size_mb,compression,number_of_spatial_chunks
dataset_id,,,,,,,,,,,,,,,,,
cmip6-kerchunk,XarrayTileTest,s3://nasa-eodc-data-store/test-data/cmip6-kerc...,1,tas,{'reference': True},"[[282.43, [0, 0, 0]]]",True,"[-59, 89]","[-179, 179]",3,2406.005859,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",float32,3.295898,Zlib(level=5),1.0
power_901_monthly_meteorology_utc.zarr,XarrayTileTest,s3://power-analysis-ready-datastore/power_901_...,1,TS,{},"[[544.66, [0, 1, 0]]]",False,"[-90, 90]","[-180, 179]",43,780.521484,"{'time': 492, 'lat': 25, 'lon': 25}","{'time': 492, 'lat': 361, 'lon': 576}",float64,2.346039,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",332.6976
cmip6-pds_GISS-E2-1-G_historical_tas,XarrayTileTest,s3://cmip6-pds/CMIP6/CMIP/NASA-GISS/GISS-E2-1-...,1,tas,{},"[[744.83, [0, 0, 0]]]",False,"[-89, 89]","[-178, 178]",6,97.888184,"{'time': 600, 'lat': 90, 'lon': 144}","{'time': 1980, 'lat': 90, 'lon': 144}",float32,29.663086,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",1.0
aws-noaa-oisst-feedstock_reference,XarrayTileTest,https://ncsa.osn.xsede.org/Pangeo/pangeo-forge...,1,sst,{'reference': True},"[[788.56, [0, 0, 0]]]",True,"[-89, 89]","[-179, 179]",4,29750.097656,"{'time': 1, 'zlev': 1, 'lat': 720, 'lon': 1440}","{'time': 15044, 'zlev': 1, 'lat': 720, 'lon': ...",int16,1.977539,Zlib(level=4),1.0
600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr,XarrayTileTest,s3://nasa-eodc-data-store/test-data/cmip6-zarr...,1,tas,{},"[[863.46, [0, 0, 0]]]",False,"[-59, 89]","[-179, 179]",3,2406.005859,"{'time': 29, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",float32,95.581055,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",1.0
600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr,XarrayTileTest,s3://nasa-eodc-data-store/test-data/cmip6-zarr...,1,tas,{},"[[279.96, [0, 0, 0]]]",False,"[-59, 89]","[-179, 179]",3,2406.005859,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",float32,3.295898,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",1.0
365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr,XarrayTileTest,s3://nasa-eodc-data-store/test-data/cmip6-zarr...,1,tas,{},"[[1809.23, [0, 0, 0]]]",False,"[-59, 89]","[-179, 179]",3,2406.005859,"{'time': 365, 'lat': 262, 'lon': 262}","{'time': 730, 'lat': 600, 'lon': 1440}",float32,95.577469,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",12.586679
single_chunk_store_lat1024_lon2048.zarr,XarrayTileTest,s3://nasa-eodc-data-store/test-data/fake-data/...,1,data,{},"[[623.52, [0, 0, 0]]]",False,"[-90, 90]","[-180, 180]",3,16.0,"{'time': 1, 'lat': 1024, 'lon': 2048}","{'time': 1, 'lat': 1024, 'lon': 2048}",float64,16.0,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",1.0
single_chunk_store_lat1448_lon2896.zarr,XarrayTileTest,s3://nasa-eodc-data-store/test-data/fake-data/...,1,data,{},"[[1013.31, [0, 0, 0]]]",False,"[-90, 90]","[-180, 180]",3,31.993164,"{'time': 1, 'lat': 1448, 'lon': 2896}","{'time': 1, 'lat': 1448, 'lon': 2896}",float64,31.993164,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ...",1.0
